In [ ]:
!pip install base36
!pip install encodec

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import base36
from encodec import EncodecModel
from encodec.utils import convert_audio
from PIL import Image
import numpy as np
import torchaudio
import torch
from IPython.display import Audio
import torchvision.transforms as transforms  
import matplotlib.pyplot as plt
from scipy.io import wavfile
from pathlib import Path
import os


In [ ]:
path = "/content/drive/MyDrive/techno/04 - Mouldy Soul - We Control This (v1).wav"

In [ ]:
# Instantiate a pretrained EnCodec model
model = EncodecModel.encodec_model_24khz()
model.set_target_bandwidth(3.0)

# Load and pre-process the audio waveform
wav, sr = torchaudio.load(path)
nwav = wav[0][:].unsqueeze(0) #only doing small amount of wav for testing

wav = convert_audio(nwav, sr, model.sample_rate, model.channels)
wav = wav.unsqueeze(0)

# Extract discrete codes from EnCodec
with torch.no_grad():
    encoded_frames = model.encode(wav)
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)  # [B, n_q, T]

In [ ]:
codes.shape

torch.Size([1, 4, 27429])

In [ ]:
cs = codes.squeeze()

In [ ]:
cs.shape

torch.Size([4, 437])

#Do the stuff:

In [ ]:
# tns = torch.from_numpy(nr)

In [ ]:
# cs = tns

In [ ]:
csr = cs.T

In [ ]:
print(csr.shape)

torch.Size([1238954, 4])


In [ ]:
# working encode
hexarr = [] # don't do this for large tensors, pre allocate memory, I think is a better way, all hacky, thinking out loud work
a = ''
for i,x in enumerate(csr):
  print(i)
  for y in x:
    nb = base36.dumps(int(y))
    if (y < 36):
      n = '-' + nb # pad because below 36 gives one char representation
      a = a + n
    else:
      a = a + nb
  hexarr = np.append(hexarr,a)
  a = ''
  

In [ ]:
hexarr.size

362825

In [ ]:
# hexarr

In [ ]:
np.savetxt('/content/drive/MyDrive/AI/amen-hexfile.txt', hexarr.tolist(), fmt='%s')

In [ ]:
npt = (np.loadtxt("/content/drive/MyDrive/AI/amen-hexfile.txt", dtype=str))

In [ ]:
npt

In [ ]:
loaded_hexarr = npt

In [ ]:
# loaded_hexarr??

In [ ]:
# npt.dtype, loaded_hexarr.shape

(dtype('<U8'), (437,))

In [ ]:
dcarr = []
st = ''
for x in loaded_hexarr:
  for i, y in enumerate(x):
    m = i % 2 
    if (m == 0):
      st = st + y
    elif (m == 1):
      st = st + y
      nb = base36.loads(st)
      ni = int(nb)
      if (ni < 0):
        spl = st.split("-")[1]
        nb = base36.loads(spl)
      dcarr = np.append(dcarr, nb)
      # print(nb)
      st = ''

In [ ]:
dcarr.shape

(1748,)

In [ ]:
ts = torch.from_numpy(dcarr.astype(np.int64))

In [ ]:
tsr = ts.reshape(437,4) # could be the known shape or len over 4 (or num quantisers)

In [ ]:
cs.shape

torch.Size([4, 437])

In [ ]:
np.savetxt('/content/drive/MyDrive/AI/amenraw.txt', cs)

Save codes in text file to compare file size saving

In [ ]:
cs

tensor([[  44,  610,  992,  ..., 1022, 1019,  408],
        [  90,  440,  699,  ...,  456,  648,  841],
        [ 302,  557, 1014,  ...,  958,  971, 1007],
        [ 978,  284,  149,  ...,  841,  940,  762]])

In [ ]:
rst = tsr.T

In [ ]:
rst

tensor([[  44,  610,  992,  ..., 1022, 1019,  408],
        [  90,  440,  699,  ...,  456,  648,  841],
        [ 302,  557, 1014,  ...,  958,  971, 1007],
        [ 978,  284,  149,  ...,  841,  940,  762]])

# test whether the tensors are the same

In [ ]:
torch.equal(cs,rst)

True

In [ ]:
fa = Path('/content/drive/MyDrive/AI/amenraw.txt').stat().st_size

fb = Path('/content/drive/MyDrive/AI/amen-hexfile.txt').stat().st_size

print(fa,fb)

43700 3933


In [ ]:
def calc_ratio(num1, num2):
    num1 = int(num1) # Now we are good
    num2 = int(num2) # Good, good
    ratio = (num1/num2) 
    return ratio

print ("Compression Ratio:",calc_ratio(fa, fb))

Compression Ratio: 11.11111111111111


In [ ]:
def percentage(part, whole):
  Percentage = (part)/(whole) * 100.
  return str(100-Percentage) + '%'

In [ ]:
print("Space saved:", percentage(fb,fa))

Space saved: 91.0%


In [ ]:
npt = (np.loadtxt("/content/drive/MyDrive/AI/technofiles.txt"))

In [ ]:
npt.size

4955816

In [ ]:
nr = npt.reshape(4, 1238954)

In [ ]:
nr.shape

(4, 1238954)